# titanic dataset with using  pipeline

In [79]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder  #for encoding the nominal feature data
from sklearn.preprocessing import MinMaxScaler   #for scaling
from sklearn.impute import SimpleImputer         # for missing values
from sklearn.tree import DecisionTreeClassifier  # to predict

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [80]:
df=pd.read_csv('Titanic-Dataset.csv')

In [81]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [82]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [83]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [84]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.3,random_state=42)

In [85]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [86]:
# imputation transformer
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),# 2 is a age column number
    ('imput_embarked',SimpleImputer(strategy='most_frequent'),[6])

],remainder='passthrough')

In [87]:
# one hot encoding
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6]) # handle_unknown=ignore means je koi nava dtype aje ga te fer ignore kardena onu
],remainder='passthrough')

In [88]:
# scaling
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))  # sare 10 columns te 10 columns eda hoye pehla 7 si fer ohe laya 2 remove hoye te 5 add hoge apa ohe vich drop first ni kita
])

In [89]:
# feature selection
trf4=SelectKBest(score_func=chi2,k=6)# k=? kine top de columns tuhi select karne aa

In [90]:
trf5=DecisionTreeClassifier()

# create pipeline

In [91]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [92]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('imput_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x000001AC863FE7A0>)),
                ('trf5', DecisionTreeClassifier())])

# pipeline vs make_pipeline

pipeline requires the naming of the steps,and make_pipeline does not require 

(same applies to columntransformer and make_column_transformer)

In [93]:
# alternate syntax
pipe1=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [94]:
pipe1

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('imput_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x000001AC863FE7A0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [95]:
# train
pipe1.fit(x_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('imput_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x000001AC863FE7A0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# explore pipeline

In [96]:
pipe1.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('imput_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=6, score_func=<function chi2 at 0x000001AC863FE7A0>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [97]:
pipe1.named_steps['columntransformer-1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('imput_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [98]:
pipe1.named_steps['columntransformer-1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('imput_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [99]:
pipe1.named_steps['columntransformer-1'].transformers_[0]

('impute_age', SimpleImputer(), [2])

In [100]:
pipe1.named_steps['columntransformer-1'].transformers_[0][1]

SimpleImputer()

In [101]:
pipe1.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.25635271])

# predict

In [102]:
y_pred=pipe1.predict(x_test)

In [103]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)                 # accracy tan ghat aa kyo ki apa trf4 vich feature selection kitre si jo ki data nu decrease kardendi aa so if we remove it then accuracy will increase

0.6007462686567164

# cross validation using pipeline

In [104]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe1,x_train,y_train,cv=5,scoring='accuracy')

array([0.64      , 0.664     , 0.672     , 0.63709677, 0.64516129])

# gridsearch using pipeline

In [ ]:
# gridsearchcv
params={
    'decisiontreeclassifier__max_depth':[1,2,3,4,5,None]
}

In [108]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe1,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('imput_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknow...,
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=6,
                                                    score_func=<function chi2 at 0x000001AC863FE7A0>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [109]:
grid.best_score_

np.float64(0.6516516129032258)

In [110]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 1}

# exporting the pipeline

In [111]:
# export 
import pickle
pickle.dump(pipe1,open('pipe1.pkl','wb'))